In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime

import matplotlib.pyplot as plt
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

%matplotlib inline
#from expo import expo

In [3]:
# testing periods dictionary
# periods = {365:'D',7:'D',12:'M',52:'W',4:'Q'}
# periods[12]

In [5]:
### Test the kwargs
# initialize a default dictionary 
# iterate through kwargs and replace or append. 
# test, 
# insert into the function
# test it again

'M'